In [78]:
from lsst.sqre.jirakit import basic_auth_from_file
basic_auth = basic_auth_from_file()

SERVER = "https://jira.lsstcorp.org/"
from jira import JIRA
js = JIRA(server=SERVER, basic_auth=basic_auth)

In [82]:
query = 'project = SIM AND Team = opsim AND type = Epic AND assignee is EMPTY AND createdDate >= "2015/07/01"'
issues = js.search_issues(query)
print len(issues)

50


In [83]:
rows = []
from lsst.sqre.jirakit import get_issue_links
link_types = ["Finish-to-Start Dependency", "Start-to-Start Dependency"]
for issue in issues:
    irow = []
    irow.append(issue.key)
    irow.append(issue.fields.summary)
    
    all_links = get_issue_links(issue, link_types)
    preds = []
    succs = []
    for alink in all_links:
        try:
            preds.append(js.issue(alink.outwardIssue.key).key)
        except AttributeError:
            try:
                succs.append(js.issue(alink.inwardIssue.key).key)
            except AttributeError:
                pass
            
    irow.append(" ".join(preds))
    irow.append(" ".join(succs))
    
    irow.append(issue.fields.timeoriginalestimate)
    irow.append(issue.fields.customfield_10202)
    irow.append(issue.fields.duedate)

    try:
        milestone = issue.fields.customfield_10224[0]
    except TypeError:
        milestone = "None"
    irow.append(milestone)
    
    #print(irow)
    rows.append(irow)


In [84]:
import csv
with open('socs_timeline.csv', 'w+') as csvfile:
    writer = csv.writer(csvfile)
    for row in rows:
        writer.writerow(row)